In [1]:
from os import listdir
from os.path import join
import os.path

from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchsummary import summary

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

import random

import cv2
import matplotlib.pyplot as plt

In [2]:
def plt_imshow(title='image', img=None, figsize=(8 ,5)):
    plt.figure(figsize=figsize)
 
    if type(img) == list:
        if type(title) == list:
            titles = title
        else:
            titles = []
 
            for i in range(len(img)):
                titles.append(title)
 
        for i in range(len(img)):
            if len(img[i].shape) <= 2:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_GRAY2RGB)
            else:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_BGR2RGB)
 
            plt.subplot(1, len(img), i + 1), plt.imshow(rgbImg)
            plt.title(titles[i])
            plt.xticks([]), plt.yticks([])
 
        plt.show()
    else:
        if len(img.shape) < 3:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        else:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
 
        plt.imshow(rgbImg)
        plt.title(title)
        plt.xticks([]), plt.yticks([])
        plt.show()

In [3]:
class myModel(nn.Module) :
  def __init__(self) :
    super(myModel, self).__init__()

    inputDim, targetDim, channels = 3, 82, 64

    self.layer0 = nn.Sequential(nn.Conv2d(inputDim, channels, kernel_size=3, stride=1, padding=1),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer1 = nn.Sequential(nn.Conv2d(channels, channels*2, kernel_size=3, stride=1, padding=1),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer2 = nn.Sequential(nn.Conv2d(channels*2, channels*4, kernel_size=3, stride=1, padding=1),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer3 = nn.Sequential(nn.Conv2d(channels*4, channels*4, kernel_size=3, stride=1, padding=1),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer4 = nn.Linear(channels*4, targetDim)

  def forward(self, input) :
    output = self.layer0(input)
    output = self.layer1(output)
    output = self.layer2(output)
    output = self.layer3(output)
    output = F.adaptive_avg_pool2d(output, (1,1)).view(output.size(0), -1)
    output = self.layer4(output)

    return output

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'using {device}')

using cuda


In [9]:
model = myModel()
# model = torch.load('../model/bestModel.pth', map_location=device)
model.load_state_dict(torch.load('../model/bestModel.pth', map_location=device))
model.to(device)
model.eval()

myModel(
  (layer0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer4): Linear(in_features=256, out_features=82, bias=True)
)

In [5]:
# def transform_image(img):
trans = transforms.Compose([transforms.Resize((100,100)),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                            ])

print(trans)


Compose(
    Resize(size=(100, 100), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
)


In [6]:
data_path = '../data/'
data_dir  = 'extracted_images'

In [7]:
print("")
label = {}
num = 0
for className in listdir(join(data_path, "extracted_images")) :
  print(f"{className}:{len(listdir(join(data_path, 'extracted_images', className)))}")
  label[className] = num
  num += 1


v:1558
,:1906
beta:2025
cos:2986
4:7396
forward_slash:199
prime:329
G:1692
mu:177
ascii_124:1339
{:376
Delta:137
7:2909
sin:4293
forall:45
):14355
lim:1675
lambda:109
9:3737
k:3074
pi:2332
}:377
z:5870
geq:693
A:12367
(:14294
i:5140
p:2680
f:3712
o:449
5:3545
tan:2450
T:3274
j:1536
l:1017
phi:355
C:5802
y:9340
neq:558
R:2671
]:780
M:2476
8:3068
e:3003
sum:2689
0:6914
q:1230
b:8651
!:1300
u:1269
=:13104
leq:973
div:868
-:33997
sqrt:8908
rightarrow:1703
2:26141
w:556
gamma:409
ldots:609
pm:802
X:26594
infty:1783
exists:21
lt:477
1:26520
d:4852
N:10862
gt:258
theta:2796
sigma:201
6:3118
+:25112
[:778
times:3251
log:2001
in:47
int:2742
alpha:2546
3:10909
H:1464
S:1413


In [39]:
from PIL import Image
trans = transforms.Compose([transforms.Resize((100,100)),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                            ])

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = Image.open(image_name)
    image = trans(image).float()
    # image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return image.cuda()  #assumes that you're using GPU

In [40]:
import cv2

im = cv2.imread("../data/output/2.jpg")
im = cv2.resize(im, dsize=(100, 100),interpolation=cv2.INTER_LINEAR)
image_swap = np.swapaxes(im, 0,2)
# image_swap = np.expand_dims(image_swap, axis=0)

print(image_swap.shape)

plt_imshow("title", im)

tensor = torch.from_numpy(image_swap).type(torch.cuda.FloatTensor)
pred = model(tensor)

probs = torch.nn.functional.softmax(pred, dim=1)
conf, classes = torch.max(probs, 1)

print(conf.item())
print(classes.item())
print(label[str(classes.item())])

(45, 45, 3)


In [38]:
import cv2

im = cv2.imread("../data/output/2.jpg")
im_path = "../data/output/1.jpg"

img = image_loader(im_path)

# pred = model(img)
# print(pred)
# probs = torch.nn.functional.softmax(pred, dim=1)
# conf, classes = torch.max(probs, 1)

# print(conf.item())
# print(classes.item())
# print('done')

AttributeError: 'NoneType' object has no attribute 'shape'

<Figure size 800x500 with 0 Axes>

In [31]:
trans = transforms.Compose([transforms.Resize((100,100)),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                            ])
trainset = torchvision.datasets.ImageFolder(root = data_path + 'extracted_images',
                                            transform = trans)

len(trainset.classes)

82

In [33]:
train_size = int(0.9 * len(trainset))
test_size = len(trainset) - train_size

train_data, test_data = torch.utils.data.random_split(trainset, [train_size, test_size])

train_loader = DataLoader(train_data,
                         batch_size = 16,
                         shuffle = True)

test_loader = DataLoader(test_data,
                         batch_size = 16,
                         shuffle = True)

print(f'train size = {train_size}\ntest size = {test_size}')
print('')
print(f'training data set = {len(train_loader)}\ntest data set = {len(test_loader)}')

test = iter(test_loader)
feature, label = next(test)

print(label)

train size = 338376
test size = 37598

training data set = 21149
test data set = 2350
tensor([16,  2,  5,  1, 27, 77, 47,  9, 11, 79, 72,  8, 35, 16,  3,  5])


In [59]:
test_loss, correct = 0, 0

for data, target in test_loader:
    data, target = data.to(device), target.to(device)
    output = model(data)
    test_loss += F.nll_loss(output, target, reduction='sum').item()
    pred = output.argmax(dim=1, keepdim=True)
    correct += pred.eq(target.view_as(pred)).sum().item()
    
test_loss /= len(test_loader.dataset)

print(f'Avg loss: {test_loss}, accuracy: {correct}/{len(test_loader.dataset)} ({100 * correct/len(test_loader.dataset)})')

Avg loss: -5.683505429531121, accuracy: 35668/37598 (94.8667482312889)
